In [2]:
!pip install pyspark==3.3.1


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 1.7 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 24.2 MB/s eta 0:00:00
  DEPRECATION: pyspark is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559
  Running setup.py install for pyspark ... done

[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: pip install --upgrade pip


In [3]:
!pip install --upgrade pip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 26.9 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: pip
    Found existing installation: pip 22.3.1
    Uninstalling pip-22.3.1:
      Successfully uninstalled pip-22.3.1


In [4]:
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession\
    .builder\
    .appName("twitter_transformation")\
    .getOrCreate()

your 131072x1 screen size is bogus. expect trouble


23/02/14 17:08:32 WARN Utils: Your hostname, cmatheus resolves to a loopback address: 127.0.1.1; using 172.28.105.128 instead (on interface eth0)
23/02/14 17:08:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/14 17:08:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [8]:
df = spark.read.json("../../data_lake/twitter_extract_dataScience")

In [9]:
df.show(10)

+--------------------+--------------------+--------------------+------------+
|                data|            includes|                meta|extract_date|
+--------------------+--------------------+--------------------+------------+
|[{411419084, 1616...|{[{2011-11-13T11:...|{1616922528303435...|  2023-01-22|
|[{137836534906265...|{[{2021-04-03T15:...|{1616922261675716...|  2023-01-22|
|[{1264433760, 161...|{[{2013-03-13T13:...|{1616921838034030...|  2023-01-22|
|[{156837570545650...|{[{2022-09-09T23:...|{1616921664758898...|  2023-01-22|
|[{1264433760, 161...|{[{2013-03-13T13:...|{1616921420621021...|  2023-01-22|
|[{1264433760, 161...|{[{2013-03-13T13:...|{1616921214621999...|  2023-01-22|
|[{156837570545650...|{[{2022-09-09T23:...|{1616920970450673...|  2023-01-22|
|[{1264433760, 161...|{[{2013-03-13T13:...|{1616920696248041...|  2023-01-22|
|[{156837570545650...|{[{2022-09-09T23:...|{1616920552576278...|  2023-01-22|
|[{137155121135112...|{[{2021-03-15T19:...|{1616920297390673...|

In [10]:
df.printSchema()

root
 |-- data: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- author_id: string (nullable = true)
 |    |    |-- conversation_id: string (nullable = true)
 |    |    |-- created_at: string (nullable = true)
 |    |    |-- edit_history_tweet_ids: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- in_reply_to_user_id: string (nullable = true)
 |    |    |-- lang: string (nullable = true)
 |    |    |-- public_metrics: struct (nullable = true)
 |    |    |    |-- impression_count: long (nullable = true)
 |    |    |    |-- like_count: long (nullable = true)
 |    |    |    |-- quote_count: long (nullable = true)
 |    |    |    |-- reply_count: long (nullable = true)
 |    |    |    |-- retweet_count: long (nullable = true)
 |    |    |-- text: string (nullable = true)
 |-- includes: struct (nullable = true)
 |    |-- users: array (nullable = true)
 |    |    

In [11]:
from pyspark.sql import functions as f

In [12]:
df.select(f.explode("data")).printSchema()

root
 |-- col: struct (nullable = true)
 |    |-- author_id: string (nullable = true)
 |    |-- conversation_id: string (nullable = true)
 |    |-- created_at: string (nullable = true)
 |    |-- edit_history_tweet_ids: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- id: string (nullable = true)
 |    |-- in_reply_to_user_id: string (nullable = true)
 |    |-- lang: string (nullable = true)
 |    |-- public_metrics: struct (nullable = true)
 |    |    |-- impression_count: long (nullable = true)
 |    |    |-- like_count: long (nullable = true)
 |    |    |-- quote_count: long (nullable = true)
 |    |    |-- reply_count: long (nullable = true)
 |    |    |-- retweet_count: long (nullable = true)
 |    |-- text: string (nullable = true)



In [13]:
df.select(f.explode("data")).show(10)

+--------------------+
|                 col|
+--------------------+
|{411419084, 16169...|
|{9547781709805281...|
|{1568375705456500...|
|{1347291458278080...|
|{8151691243222016...|
|{411419084, 16169...|
|{1568375705456500...|
|{1527071629, 1616...|
|{411419084, 16169...|
|{1568375705456500...|
+--------------------+
only showing top 10 rows



In [15]:
df.select(f.explode("data").alias("tweets"))\
.select("tweets.author_id", "tweets.conversation_id",
        "tweets.created_at", "tweets.id",
        "tweets.public_metrics.*", "tweets.text").show()

+-------------------+-------------------+--------------------+-------------------+----------------+----------+-----------+-----------+-------------+--------------------+
|          author_id|    conversation_id|          created_at|                 id|impression_count|like_count|quote_count|reply_count|retweet_count|                text|
+-------------------+-------------------+--------------------+-------------------+----------------+----------+-----------+-----------+-------------+--------------------+
|          411419084|1616922528303435776|2023-01-21T22:15:...|1616922528303435776|               0|         0|          0|          0|            1|RT @Hackwithgibso...|
| 954778170980528129|1616922520447221762|2023-01-21T22:15:...|1616922520447221762|               0|         0|          0|          0|           22|RT @CatherineAden...|
|1568375705456500738|1616922500461481988|2023-01-21T22:15:...|1616922500461481988|              32|         0|          0|          0|            1|I 

In [16]:
tweet_df = df.select(f.explode("data").alias("tweets"))\
  .select("tweets.author_id", "tweets.conversation_id",
        "tweets.created_at", "tweets.id",
        "tweets.public_metrics.*", "tweets.text")

In [18]:
tweet_df.show(5)

+-------------------+-------------------+--------------------+-------------------+----------------+----------+-----------+-----------+-------------+--------------------+
|          author_id|    conversation_id|          created_at|                 id|impression_count|like_count|quote_count|reply_count|retweet_count|                text|
+-------------------+-------------------+--------------------+-------------------+----------------+----------+-----------+-----------+-------------+--------------------+
|          411419084|1616922528303435776|2023-01-21T22:15:...|1616922528303435776|               0|         0|          0|          0|            1|RT @Hackwithgibso...|
| 954778170980528129|1616922520447221762|2023-01-21T22:15:...|1616922520447221762|               0|         0|          0|          0|           22|RT @CatherineAden...|
|1568375705456500738|1616922500461481988|2023-01-21T22:15:...|1616922500461481988|              32|         0|          0|          0|            1|I 

dados dos usuarios

In [21]:
df.select(f.explode("includes.users")).show()

+--------------------+
|                 col|
+--------------------+
|{2011-11-13T11:40...|
|{2018-01-20T18:10...|
|{2022-09-09T23:08...|
|{2021-01-07T21:18...|
|{2016-12-31T12:13...|
|{2013-06-18T07:35...|
|{2021-04-03T15:15...|
|{2010-12-01T23:49...|
|{2020-02-01T18:34...|
|{2011-11-13T11:40...|
|{2014-11-11T19:28...|
|{2022-09-09T23:08...|
|{2014-05-16T16:56...|
|{2013-03-13T13:19...|
|{2021-10-14T14:41...|
|{2013-03-13T13:19...|
|{2011-11-13T11:40...|
|{2022-09-09T23:08...|
|{2008-08-22T05:41...|
|{2017-04-01T07:56...|
+--------------------+
only showing top 20 rows



In [22]:
df.select(f.explode("includes.users").alias("users")).show(5)

+--------------------+
|               users|
+--------------------+
|{2011-11-13T11:40...|
|{2018-01-20T18:10...|
|{2022-09-09T23:08...|
|{2021-01-07T21:18...|
|{2016-12-31T12:13...|
+--------------------+
only showing top 5 rows



In [23]:
df.select(f.explode("includes.users").alias("users")).select("users.*").printSchema()

root
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- username: string (nullable = true)



In [24]:
user_df = df.select(f.explode("includes.users").alias("users")).select("users.*")

In [26]:
user_df.show()

+--------------------+-------------------+--------------------+---------------+
|          created_at|                 id|                name|       username|
+--------------------+-------------------+--------------------+---------------+
|2011-11-13T11:40:...|          411419084|               Rahul|  rahul05ranjan|
|2018-01-20T18:10:...| 954778170980528129|  Frédéric Guariento| Analogique2018|
|2022-09-09T23:08:...|1568375705456500738|    Hack With Gibson|Hackwithgibson1|
|2021-01-07T21:18:...|1347291458278080513|1000 Days Of Codi...|1000dayscodingb|
|2016-12-31T12:13:...| 815169124322201600|        Data science|  Datascience__|
|2013-06-18T07:35:...|         1527071629|         DAOUD Amine|   DAOUDAMINE88|
|2021-04-03T15:15:...|1378365349062656001|⚓AnaBeq.OTRL 💧| ...|  anass_forever|
|2010-12-01T23:49:...|          221916197|        Henry Taquez|       hetaquez|
|2020-02-01T18:34:...|1223675171418099713|    Ramsey Elbasheer| genericgranola|
|2011-11-13T11:40:...|          411419084

In [27]:
tweet_df.coalesce(1).write.mode("overwrite").json('output/tweet')
user_df.coalesce(1).write.mode("overwrite").json('output/user')

presta atenção nos tamanhos dos arquivos caso arquivos com uma grande massa de dados, salvar em diversos arquivos